In [ ]:
%%capture
!pip install selenium

In [ ]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=5b670db334ac43f52f2956a3348875b8b88cfa5ed3a40390c3392d8c8d366944
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=37&\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2012-03-01&'\
        + 'modified%2Cto=2022-12-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     save_csv = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_csv('checkpoint_table.csv')
                print('Checkpoint saved!')
                save_counter = 0

        if save_csv:
            out.to_csv("society_lenta_{}_{}.csv".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [ ]:
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "1"
bloc = "1" #общество/россия 1 экономика 4 забота о себе 87 бывший ссср 3 наука 5 спорт 8 силовые структуры (происшествия) 37 путешестия 48
dateFrom = '2012-03-01'
dateTo = "2023-12-22"

if use_parser == "LentaRu":
    param_dict = {'query'     : query,
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material,
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2012-03-01', 'dateTo': '2023-12-22', 'sort': '3', 'title_only': '0', 'type': '1', 'bloc': '1', 'domain': '1'}


In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5,
                         save_csv = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Parsing articles from 2023-08-11 to 2023-09-17
Parsing articles from 2023-09-18 to 2023-10-25
Parsing articles from 2023-10-26 to 2023-12-02
Parsing articles from 2023-12-03 to 2023-12-22
Finish
103382


,rightcol,docid,lastmodtime,part,title,type,url,tags,bloc,domain,modified,image_url,text,status,pubdate,snippet
0,Мавзолей Ленина традиционно закрывают на профи...,24715,1670838205,0,Мавзолей Ленина закроют на профилактику,1,https://lenta.ru/news/2012/03/01/close/,[],1,1,1330564249,https://icdn.lenta.ru/images/0000/0028/0000002...,"Мавзолей Ленина. Фото ""Ленты.Ру"" Мавзолей, в к...",0,1330564249,"Мавзолей Ленина. Фото ""Ленты.Ру"" Мавзолей, в к..."
1,Навальный назвал борьбу с коррупцией в РФ сущн...,24718,1670838205,0,Навальный опубликовал свою экономическую прогр...,1,https://lenta.ru/news/2012/03/01/navalny/,[],1,1,1330568367,https://icdn.lenta.ru/images/0000/0028/0000002...,"Алексей Навальный. Фото РИА Новости, Григорий ...",0,1330568367,"Алексей Навальный. Фото РИА Новости, Григорий ..."
2,"Московские власти заявили, что ""никаких палато...",24760,1670838208,0,"Собянин пообещал не допустить ""майдана"" в Москве",1,https://lenta.ru/news/2012/03/01/maidan/,[],1,1,1330592718,https://icdn.lenta.ru/images/0000/0028/0000002...,"Сергей Собянин. Фото РИА Новости, Евгений Сама...",0,1330592718,"Сергей Собянин. Фото РИА Новости, Евгений Сама..."
3,Правительство и Кремль решили не проводить Оли...,24744,1670838207,0,Россия отказалась от юношеской Олимпиады в Даг...,1,https://lenta.ru/news/2012/03/01/dagestan/,[],1,1,1330593848,https://icdn.lenta.ru/images/0000/0028/0000002...,"В горах Дагестана. Фото ""Кавказский узел"" Росс...",0,1330593848,"В горах Дагестана. Фото ""Кавказский узел"" Росс..."
4,В преступную группу входили три человека,24782,1670838209,0,В Москве разоблачили сбытчиков поддельных бенз...,1,https://lenta.ru/news/2012/03/01/saw/,[],1,1,1330595952,https://icdn.lenta.ru/images/0000/0028/0000002...,Фото с сайта kaskadtools.ru В Москве сотрудник...,0,1330595952,Фото с сайта kaskadtools.ru В Москве ... на са...


In [ ]:
tbl.loc[tbl['bloc']== 49, ['bloc', 'text', 'url']]

,bloc,text,url
858,49,Кадр: Joe HaTTab / YouTube Трэвел-блогер из Об...,https://lenta.ru/news/2020/01/09/depressed/
862,49,Поселок Эгвекинот Фото: Андрей Шапран / РИА Н...,https://lenta.ru/news/2020/01/09/found/
892,49,Фото: «Таймырский телеграф» Археологи нашли на...,https://lenta.ru/news/2020/01/09/chess/
1377,49,Архивное фото Фото: страница «Большая арктичес...,https://lenta.ru/news/2020/02/10/trip/
1388,49,Фото: Павел Львов / РИА Новости Минприроды соб...,https://lenta.ru/news/2020/02/10/bears/
...,...,...,...
36703,49,Фото: Сергей Карпухин / Reuters Марк Леонов В ...,https://lenta.ru/news/2023/10/02/stado/
36780,49,Фото: Unsplash Варвара Родикова В Момском райо...,https://lenta.ru/news/2023/10/02/babochka/
36867,49,Фото: Антон Вергун / РИА Новости Сергей Истоми...,https://lenta.ru/news/2023/10/02/deeps/
37227,49,Фото: Pelivanov Yuriy / Wikimedia Commons Марк...,https://lenta.ru/news/2023/11/07/svyaschennik-...


In [ ]:
tbl.head()

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1205238,https://lenta.ru/news/2022/02/23/heart/,Названы продукты для здоровья сердца,1645567200,1648746997,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/23/00/202...,1645567200,Фото: Pixabay.com Марина Совина Употребление н...,Названы продукты для здоровья сердца,"Фото: Pixabay.com Марина Совина ... диетологи,..."
1,1205325,https://lenta.ru/news/2022/02/23/ice_cream/,Россиянам развеяли популярный миф о лечении бо...,1645590960,1648747025,1,1,0,0,87,[355],https://icdn.lenta.ru/images/2022/02/23/06/202...,1645590960,Фото: Pixabay Марина Совина Врач-отоларинголог...,Россиянам развеяли популярный миф о лечении бо...,Фото: Pixabay Марина Совина Врач-... лекарство...
2,1205331,https://lenta.ru/news/2022/02/23/diabet/,Названы способные предотвратить диабет продукты,1645592820,1648747044,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/23/08/202...,1645592820,"Фото: Pixabay Марина Совина Семейный врач, спе...",Названы способные предотвратить диабет продукты,Фото: Pixabay Марина Совина Семейный ... вспят...
3,1205841,https://lenta.ru/news/2022/02/25/broccoli/,Названо средство для крепкого сна даже после ч...,1645769520,1648747186,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/24/12/202...,1645769520,Фото: Louis Hansel / Unsplash Олег Парамонов А...,Названо средство для крепкого сна даже после ч...,Фото: Louis Hansel / Unsplash Олег ... крепко ...
4,1205881,https://lenta.ru/news/2022/02/25/4veg/,Диетолог рассказала о вредном воздействии на м...,1645776720,1648747198,1,1,0,0,87,[354],https://icdn.lenta.ru/images/2022/02/24/12/202...,1645776720,Фото: Shutterstock Никита Савин Диетолог из СШ...,Диетолог рассказала о вредном воздействии на м...,Фото: Shutterstock Никита Савин Диетолог ... о...
